<a href="https://colab.research.google.com/github/mkeohane01/GAN-Image-Denoising/blob/srgan-testing/notebooks/training_SRGAN_mbk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
if not os.path.exists('srgan.pth.tar'):
    !pip install -q torch_snippets
    !wget -q https://raw.githubusercontent.com/sizhky/a-PyTorch-Tutorial-to-Super-Resolution/master/models.py -O models.py
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    downloaded = drive.CreateFile({'id': '1_PJ1Uimbr0xrPjE8U3Q_bG7XycGgsbVo'})
    downloaded.GetContentFile('srgan.pth.tar')
    from torch_snippets import *
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 74.9 MB/s eta 0

In [4]:
!pip install kaggle
from google.colab import files
files.upload()

# Download kaggle dataset
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Small dataset
!kaggle datasets download -d rajat95gupta/smartphone-image-denoising-dataset

# Medium dataset
#!kaggle datasets download -d soumikrakshit/smartphone-image-denoising-dataset

Saving kaggle.json to kaggle.json
100% 6.16G/6.16G [02:37<00:00, 42.9MB/s]
100% 6.16G/6.16G [02:37<00:00, 41.9MB/s]


In [5]:
# Unzip the smartphone denoising dataset
!unzip '/content/smartphone-image-denoising-dataset.zip'

Archive:  /content/smartphone-image-denoising-dataset.zip
  inflating: SIDD_Small_sRGB_Only/Data/0001_001_S6_00100_00060_3200_L/GT_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0001_001_S6_00100_00060_3200_L/NOISY_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0002_001_S6_00100_00020_3200_N/GT_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0002_001_S6_00100_00020_3200_N/NOISY_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0003_001_S6_00100_00060_3200_H/GT_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0003_001_S6_00100_00060_3200_H/NOISY_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0004_001_S6_00100_00060_4400_L/GT_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0004_001_S6_00100_00060_4400_L/NOISY_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0005_001_S6_00100_00060_4400_N/GT_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0005_001_S6_00100_00060_4400_N/NOISY_SRGB_010.PNG  
  inflating: SIDD_Small_sRGB_Only/Data/0006_001

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torch_snippets import *
from models import Generator, Discriminator
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import transforms as T


/content/models.py:255: SyntaxWarning: "is" with a literal. Did you mean "=="?
  out_channels = (n_channels if i is 0 else in_channels * 2) if i % 2 is 0 else in_channels
/content/models.py:255: SyntaxWarning: "is" with a literal. Did you mean "=="?
  out_channels = (n_channels if i is 0 else in_channels * 2) if i % 2 is 0 else in_channels
/content/models.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  stride=1 if i % 2 is 0 else 2, batch_norm=i is not 0, activation='LeakyReLu'))
/content/models.py:258: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  stride=1 if i % 2 is 0 else 2, batch_norm=i is not 0, activation='LeakyReLu'))


In [7]:
preprocess_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225]),
    T.Resize((256, 256)),
])

postprocess_transform = T.Compose([
    T.Resize((256, 256)),
    T.ToPILImage(),
])


class ImagePairDataset(Dataset):
    def __init__(self, noisy_image_paths, sharp_image_paths, transform=None):
        """
        Args:
            noisy_image_paths (list of str): Paths to the noisy images.
            sharp_image_paths (list of str): Paths to the sharp, high-definition images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        assert len(noisy_image_paths) == len(sharp_image_paths), "The number of noisy and sharp images must be the same."
        self.noisy_image_paths = noisy_image_paths
        self.sharp_image_paths = sharp_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, idx):
        noisy_image = Image.open(self.noisy_image_paths[idx]).convert('RGB')
        sharp_image = Image.open(self.sharp_image_paths[idx]).convert('RGB')

        if self.transform:
            noisy_image = self.transform(noisy_image)
            sharp_image = self.transform(sharp_image)

        return noisy_image, sharp_image


In [21]:

# Function to load image pairs from a folder
def load_image_pairs(folder):
    gt_images = []
    noisy_images = []
    for root, _, files in os.walk(folder):
        for file in files:
            if 'GT' in file:
                gt_images.append(os.path.join(root, file))
            elif 'NOISY' in file:
                noisy_images.append(os.path.join(root, file))
    return gt_images, noisy_images

# Function to train the SRGAN model
def train_srgan_model(generator, discriminator, train_loader, val_loader, criterion_g, criterion_d, optimizer_g, optimizer_d, num_epochs=10):
    # Loop through each epoch
    for epoch in range(num_epochs):
        # Set both the generator and discriminator to training mode
        generator.train()
        discriminator.train()

        # Initialize running losses for both generator and discriminator
        running_loss_g = 0.0
        running_loss_d = 0.0

        # Loop through the dataset
        for i, (gt_image, noisy_image) in enumerate(train_loader):
            # Move images to the appropriate device (GPU or CPU)
            gt_image = gt_image.to(device)
            noisy_image = noisy_image.to(device)

            # Zero the gradients for both optimizers
            optimizer_g.zero_grad()
            optimizer_d.zero_grad()

            # Generate super-resolved images from noisy images
            sr_image = generator(noisy_image)
            # print(sr_image.shape)
            # resize the output image with the transform
            sr_image = T.Resize((256, 256))(sr_image)
            # display image
            # PIL_image = transforms.ToPILImage()(sr_image[0])
            # display(PIL_image)
            # print(sr_image.shape)
            # Train the discriminator
            # Discriminator's goal is to distinguish between real (gt_image) and fake (sr_image) images

            # Calculate discriminator loss on real images
            real_output = discriminator(gt_image)
            loss_d_real = criterion_d(real_output, torch.ones_like(real_output))

            # Calculate discriminator loss on fake images
            fake_output = discriminator(sr_image.detach())  # Detach to avoid training G on these labels
            loss_d_fake = criterion_d(fake_output, torch.zeros_like(fake_output))

            # Total discriminator loss
            loss_d = (loss_d_real + loss_d_fake) / 2

            # Backpropagation for discriminator
            loss_d.backward()
            optimizer_d.step()

            # Train the generator
            # Generator's goal is to fool the discriminator into thinking the generated images are real

            # Calculate generator loss
            output = discriminator(sr_image.detach())
            loss_g = criterion_g(output, torch.ones_like(output)) + criterion_g(sr_image, gt_image)

            # Backpropagation for generator
            loss_g.backward()
            optimizer_g.step()

            # Update running losses
            running_loss_g += loss_g.item() * noisy_image.size(0)
            running_loss_d += loss_d.item() * noisy_image.size(0)

            # Print progress
            if i % 4 == 0:
              print(f'Epoch {epoch+1}/{num_epochs}, Batch {i}/{len(train_loader)}, Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}')


        # Calculate average losses for the current epoch
        epoch_loss_g = running_loss_g / len(train_loader.dataset)
        epoch_loss_d = running_loss_d / len(train_loader.dataset)

        # Print losses for the current epoch
        print(f'Train - Epoch {epoch+1}/{num_epochs}, Generator Loss: {epoch_loss_g:.4f}, Discriminator Loss: {epoch_loss_d:.4f}')

        # Compute metrics on validation set
        sr_images = []
        gt_images = []
        input_images = []
        for gt_image, noisy_image in val_loader:
            gt_image = gt_image.to(device)
            noisy_image = noisy_image.to(device)
            sr_image = generator(noisy_image)
            sr_image = T.Resize((256, 256))(sr_image)
            sr_images.append(sr_image.detach())
            input_images.append(noisy_image.detach())
            gt_images.append(gt_image.detach())

        # Compute and print metrics
        psnr = compute_psnr(torch.cat(sr_images), torch.cat(gt_images))
        ssim = compute_ssim(torch.cat(sr_images), torch.cat(gt_images))
        fid = compute_fid(torch.cat(sr_images), torch.cat(gt_images))
        print(f'Val - PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, FID: {fid:.4f}')

        # Display the 3 images
        sr_image = sr_images[0]
        input_image = input_images[0]
        gt_image = gt_images[0]
        PIL_sr_image = T.ToPILImage()(sr_image[0])
        PIL_input_image = T.ToPILImage()(input_image[0])
        PIL_gt_image = T.ToPILImage()(gt_image[0])
        display(PIL_sr_image)
        display(PIL_input_image)
        display(PIL_gt_image)

    return generator, discriminator



# Function to compute PSNR
def compute_psnr(sr_image, gt_image):
    mse = nn.MSELoss()(sr_image, gt_image)
    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
    return psnr.item()

# Function to compute SSIM
def compute_ssim(sr_image, gt_image):
    ssim_val = ssim(sr_image, gt_image, data_range=1.0, win_size=11, win_sigma=1.5, k1=0.01, k2=0.03)
    return ssim_val.item()

# Function to compute FID
def compute_fid(sr_images, gt_images):
    fid_score = calculate_fid(sr_images, gt_images)
    return fid_score

In [14]:
# Main function to load data, train model, and compute metrics
def main(train_folder, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Load image pairs
    gt_images, noisy_images = load_image_pairs(train_folder)
    print(f'Number of training images: {len(gt_images)}')
    print(f'Number of validation images: {len(noisy_images)}')

    # split to train and val
    noisy_train_images, noisy_val_images = train_test_split(noisy_images, test_size=0.2, random_state=42)
    gt_train_images, gt_val_images = train_test_split(gt_images, test_size=0.2, random_state=42)

    # Create dataset
    train_set = ImagePairDataset(noisy_train_images, gt_train_images, transform=preprocess_transform)
    val_set = ImagePairDataset(noisy_val_images, gt_val_images, transform=preprocess_transform)

    # Create data loaders
    train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=2, shuffle=False)

    # Initialize model
    gen = Generator().to(device)
    disc = Discriminator().to(device)
    # print(gen)
    # print(disc)

    # Define loss function and optimizer
    criterion_g = nn.MSELoss() # minimize difference between generated and real images
    criterion_d = nn.BCEWithLogitsLoss()  # For discriminator, to classify between real and fake images

    # Optimizers for generator and discriminator
    optimizer_g = optim.Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(disc.parameters(), lr=0.0002, betas=(0.5, 0.999))

    num_epochs = 5

    # Train the model
    generator, discriminator = train_srgan_model(gen, disc, train_loader, val_loader, criterion_g, criterion_d, optimizer_g, optimizer_d, num_epochs)

    # Save model
    torch.save(generator.state_dict(), 'generator.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')


In [27]:
# Define variables
train_folder = '/content/SIDD_Small_sRGB_Only/Data'  # Path to the folder containing image pairs
num_epochs = 2

# Call the main function
main(train_folder, num_epochs)

Using device: cuda

Number of training images: 160

Number of validation images: 160

Epoch 1/5, Batch 0/32, Loss D: 0.6850, Loss G: 33.6290

Epoch 1/5, Batch 4/32, Loss D: 0.4994, Loss G: 13.8914

...

Epoch 1/5, Batch 20/32, Loss D: 0.3532, Loss G: 21.5467

Epoch 1/5, Batch 24/32, Loss D: 0.1200, Loss G: 78.6021

Epoch 1/5, Batch 28/32, Loss D: 0.1488, Loss G: 86.5832

Train - Epoch 1/5, Generator Loss: 48.1438, Discriminator Loss: 0.7213

NameError: name 'ssim' is not defined

In [26]:
import gc
gc.collect()
torch.cuda.empty_cache()